In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Dense, Dropout, Flatten, Reshape, LSTM, BatchNormalization
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
import numpy as np
import os

: 

In [ ]:
!pip install pyedflib

In [ ]:
import pyedflib
import numpy as np

def _read_py_function(filename, num_channels=16):
    # Open the EDF file
    f = pyedflib.EdfReader(filename)
    
    # Get the number of channels and the signal labels
    n_channels = f.signals_in_file
    print(f"Total channels in file: {n_channels}")
    
    # Ensure that we are fetching only the desired number of channels (16 in this case)
    if num_channels > n_channels:
        raise ValueError(f"The file contains only {n_channels} channels, but {num_channels} were requested.")
    
    # Initialize eeg_data to store only the first `num_channels`
    eeg_data = np.zeros((num_channels, f.getNSamples()[0]), dtype=np.float32)
    
    # Read only the first `num_channels` channels
    for i in range(num_channels):
        eeg_data[i, :] = f.readSignal(i)

    n_samples = f.getNSamples()[0]
    reminder = int(n_samples % 160)
    
    # Print statement to check values
    print(f"Original n_samples: {n_samples}")
    print(f"Reminder: {reminder}")
    
    n_samples -= reminder
    seconds = int(n_samples / 160)
    
    # Extract person_id from the filename
    path = filename.split("\\")
    person_id = int(path[-1].partition("S")[2].partition("R")[0])
    
    # Create one-hot encoded labels
    label = np.zeros(10, dtype=bool)
    label[person_id-1] = 1
    labels = np.tile(label, (seconds, 1))
    
    # Normalization step
    for i in range(num_channels):
        channel_data = eeg_data[i, :]
        mean_i = np.mean(channel_data)  # Compute mean of channel i
        std_i = np.std(channel_data)    # Compute standard deviation of channel i
        
        if std_i != 0:  # Avoid division by zero
            eeg_data[i, :] = (channel_data - mean_i) / std_i  # Normalize channel i

    # Transpose the data to shape (n_samples, n_channels)
    eeg_data = eeg_data.T
    
    if reminder > 0:
        eeg_data = eeg_data[:-reminder, :]
    
    # Split the data into 160-sample chunks
    intervals = np.linspace(0, n_samples, num=seconds, endpoint=False, dtype=int)
    eeg_data = np.split(eeg_data, intervals)
    del eeg_data[0]
    eeg_data = np.array(eeg_data)
    
    return eeg_data, labels


In [ ]:
import os
import numpy as np
from sklearn.model_selection import train_test_split

def load_eeg_data_with_pyedflib(data_path, max_subjects=10):
    # Initialize the global variables
    global subject_data, all_train_data, all_train_labels, all_test_data, all_test_labels
    global train_data, val_data, train_labels, val_labels

    # Initialize the variables before using them
    subject_data = {}
    all_train_data = []
    all_train_labels = []
    all_test_data = []
    all_test_labels = []

    subject_count = 0  # Counter to track the number of subjects processed

    for subject_dir in os.listdir(data_path):
        subject_path = os.path.join(data_path, subject_dir)
        if os.path.isdir(subject_path):
            print(f"Processing subject: {subject_dir}")
            subject_data[subject_dir] = {}
            for recording_file in os.listdir(subject_path):
                recording_path = os.path.join(subject_path, recording_file)
                if recording_file.endswith('.edf'):
                    recording_id = os.path.splitext(recording_file)[0]
                    print(f"Loading recording: {recording_file}")

                    try:
                        #eeg_data, labels = _read_py_function(recording_path)
                        eeg_data, labels = _read_py_function(recording_path)

                        print(f"Data shape after processing: {eeg_data.shape}")
                        print(f"Labels shape: {labels.shape}")

                        train_data, test_data, train_labels, test_labels = train_test_split(
                            eeg_data, labels, test_size=0.1, random_state=42
                        )
                        
                        all_train_data.append(train_data)
                        all_train_labels.append(train_labels)
                        all_test_data.append(test_data)
                        all_test_labels.append(test_labels)

                    except Exception as e:
                        print(f"Error loading {recording_file}: {e}")

            subject_count += 1  # Increment the subject counter

            # Stop processing once the desired number of subjects is reached
            if subject_count >= max_subjects:
                print(f"Processed {max_subjects} subjects. Stopping further processing.")
                break

    print(f"Train data list length: {len(all_train_data)}")
    print(f"Train labels list length: {len(all_train_labels)}")
    print(f"Test data list length: {len(all_test_data)}")
    print(f"Test labels list length: {len(all_test_labels)}")

    if all_train_data and all_train_labels:
        all_train_data = np.concatenate(all_train_data, axis=0)
        all_train_labels = np.concatenate(all_train_labels, axis=0)
    else:
        print("No training data loaded.")
        return None, None, None, None, None, None

    if all_test_data and all_test_labels:
        all_test_data = np.concatenate(all_test_data, axis=0)
        all_test_labels = np.concatenate(all_test_labels, axis=0)
    else:
        print("No test data loaded.")
        return None, None, None, None, None, None

    if len(all_train_data) == 0 or len(all_train_labels) == 0:
        print("No data available for splitting into training and validation sets.")
        return None, None, None, None, None, None

    # Split train data further into training and validation sets
    train_data, val_data, train_labels, val_labels = train_test_split(
        all_train_data, all_train_labels, test_size=0.25, random_state=42
    )

    return train_data, train_labels, val_data, val_labels, all_test_data, all_test_labels


In [ ]:
data_path = r"C:\Users\akhileshsing2024\Downloads\eeg-motor-movementimagery-dataset-1.0.0\files"
train_data, train_labels, val_data, val_labels, test_data, test_labels = load_eeg_data_with_pyedflib(data_path)
# Access global variables directly if needed
print(f"Train data shape: {train_data.shape if train_data is not None else 'None'}")
print(f"Validation data shape: {val_data.shape if val_data is not None else 'None'}")
print(f"Test data shape: {test_data.shape if test_data is not None else 'None'}")

In [ ]:
LSTM:

import tensorflow as tf

def eeg_biometric_identification_model_lstm(input_shape, n_classes, lstm_size=192, keep_prob=0.5):
    inputs = tf.keras.Input(shape=input_shape)

    # Initial LSTM Layers for feature extraction (replacing Conv1D layers)
    lstm1 = tf.keras.layers.LSTM(units=128, return_sequences=True)(inputs)  # First LSTM layer
    lstm2 = tf.keras.layers.LSTM(units=256, return_sequences=True)(lstm1)   # Second LSTM layer
    lstm3 = tf.keras.layers.LSTM(units=512, return_sequences=True)(lstm2)   # Third LSTM layer
    lstm4 = tf.keras.layers.LSTM(units=1024, return_sequences=False)(lstm3) # Fourth LSTM layer

    # Fully connected layer with 192 neurons and dropout
    fc1 = tf.keras.layers.Dense(units=192, activation='relu')(lstm4)
    dropout_fc1 = tf.keras.layers.Dropout(rate=1 - keep_prob)(fc1)

    # Reshape to (batch_size, timesteps=1, features=192) for additional LSTM layers
    lstm_input = tf.keras.layers.Reshape((1, 192))(dropout_fc1)

    # Add the two specific LSTM layers from the original architecture
    lstm_out = tf.keras.layers.LSTM(units=lstm_size, return_sequences=True)(lstm_input)  # First LSTM layer
    lstm_out = tf.keras.layers.LSTM(units=lstm_size, return_sequences=False)(lstm_out)  # Second LSTM layer

    # Fully connected layers after LSTM
    fc2 = tf.keras.layers.Dense(units=192, activation='relu')(lstm_out)
    fc3 = tf.keras.layers.Dense(units=192, activation='relu')(fc2)

    # Output layer (Softmax for classification)
    output = tf.keras.layers.Dense(n_classes, activation='softmax')(fc3)

    # Create the model
    model = tf.keras.Model(inputs=inputs, outputs=output)

    return model

# Example usage
input_shape = (160, 64)  # 160 samples, 64 channels (1-second EEG recording)
n_classes = 10  # Number of subjects
keep_prob = 0.5  # Dropout rate

model = eeg_biometric_identification_model_lstm(input_shape, n_classes, lstm_size=192, keep_prob=keep_prob)
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

# Summary
model.summary()

In [ ]:
checkpoint_path = "C:/Users/akhileshsing2024/Desktop/Epochs/best_model/LSTM_16.weights.h5"  # Save the best model in the current directory
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 save_best_only=True,  # Save only the best model
                                                 monitor='val_loss',  # Monitor validation loss
                                                 mode='min',  # Save the model with the lowest validation loss
                                                 verbose=1)

# Training the model
with tf.device('/GPU:0'):  # Ensure training uses the GPU
    history = model.fit(train_data, train_labels, 
                        epochs=20, 
                        validation_data=(val_data, val_labels), 
                        batch_size=80,  
                        callbacks=[cp_callback]) 

In [ ]:
# Save the training history as a numpy file
np.save("history_LSTM_16.npy", history.history)

# Load the training history
history = np.load("history_LSTM_16.npy", allow_pickle=True).item()

# Ensure history is a dictionary
if type(history) is not dict:
    history = history.history

# Find the epoch with the highest validation accuracy
max_value = max(history['val_accuracy'])
print(f"Max Validation Accuracy: {max_value}")

max_index = history['val_accuracy'].index(max_value)
print(f"Best Epoch Index: {max_index}")
print(f"Corresponding Training Accuracy: {history['accuracy'][max_index]}")

# Load the best checkpoint based on validation accuracy
model.load_weights(checkpoint_path)

# Evaluate the model on the test data
test_loss, test_accuracy = model.evaluate(x=test_data, y=test_labels)
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")

In [ ]:
# Plot training and validation accuracy
import matplotlib.pyplot as plt
plt.figure(figsize=(8, 6))
plt.plot(history['accuracy'], label='Training Accuracy')
plt.plot(history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')
#plt.savefig("accuracy_plot.png")  # Save the accuracy plot
plt.show()

# Plot training and validation loss
plt.figure(figsize=(8, 6))
plt.plot(history['loss'], label='Training Loss')
plt.plot(history['val_loss'], label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
#plt.savefig("loss_plot.png")  # Save the loss plot
plt.show()

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import numpy as np

# Get the predictions from the model on the test data
predictions = model.predict(test_data)

# Convert the predicted probabilities into class predictions (argmax over class probabilities)
predicted_classes = np.argmax(predictions, axis=1)

# Convert the one-hot encoded test labels back to their original class labels
true_classes = np.argmax(test_labels, axis=1)

# Calculate the confusion matrix
cm = confusion_matrix(true_classes, predicted_classes)

# Print confusion matrix
print(f"Confusion Matrix:\n{cm}")

# Plot the confusion matrix
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=np.arange(n_classes))
disp.plot(cmap='Blues', values_format='d')
plt.title('Confusion Matrix')
plt.show()